#### This is a modified version for file 'Step 2 - Collect and process data from strava--HR, speed, altitude data'.
It can be used when we want to analyse data from users that have used other GPS sensors (instead of using their smartphone's GPS and connecting it to Strava for collecting GPS data) for recording their movement

In [1]:
import sys
# Put here the directory containing the folder where the definitions are saved
sys.path.insert(0, r'C:\Users\demdr\UTS\Important files for PhD thesis\Definitions')


In [2]:
from definitions_for_scraping_fitbit_data import scrape_movement_data
from definitions_for_altitude_data_processing import create_firstgradediff_forAlt,find_altitude_features
from definitions_for_speed_data_processing import basic_speed_functions
from definitions_for_hr_data_processing import add_smoother_HR,create_firstgradediff_forHR,find_HR_mean_and_std_thirtyseconds,detect_faulty_hr_secondstep,calculate_intensity_of_HR
from definitions_for_HR_peak_detection import HR_peak_detection

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pyproj import Proj
from pyproj import Transformer
import datetime as dt

import folium
from IPython.display import IFrame
from IPython.display import display
from folium.plugins import HeatMap
import branca.colormap

In [3]:
participant_ID = '07'
participant_ID

'07'

In [8]:
this_path = r'C:\Users\demdr\Desktop\Testing the thesis functions\Project data\Raw data\Participants\ID\GPS data\GPS_data_5s_marked.csv'
this_path = this_path.replace('ID',participant_ID)
df = pd.read_csv(this_path)
df

inProj  = Proj("EPSG:21781",preserve_units=True)
outProj = Proj("EPSG:4326") 

transformer = Transformer.from_crs("epsg:21781", "epsg:4326")

In [9]:
df['Latitude']=0
df['Longitude']=0
for i in df.index:
    this_x = df.loc[i, 'X']
    this_y = df.loc[i, 'Y']
    y1,x1 =  (this_y, this_x)
    df.loc[i, 'Latitude']=transformer.transform(x1,y1)[0]
    df.loc[i, 'Longitude']=transformer.transform(x1,y1)[1]
df

,Unnamed: 0,time,X,Y,Sound,Dust,TempEN,RH,Light,Latitude,Longitude
0,0,2016-04-07 14:25:13.250,681400.343979,247357.081757,48.328609,0.033226,15.8204,23.2496,430.48,47.372025,8.516394
1,1,2016-04-07 14:25:13.000,681400.343979,247357.081757,48.328609,0.033226,15.8204,23.2496,430.48,47.372025,8.516394
2,2,2016-04-07 14:25:18.000,681400.343979,247357.081757,48.328609,0.033226,15.8204,23.2496,430.48,47.372025,8.516394
3,3,2016-04-07 14:25:23.000,681400.343979,247357.081757,48.328609,0.033226,15.8204,23.2496,430.48,47.372025,8.516394
4,4,2016-04-07 14:25:28.000,681400.343979,247357.081757,48.328609,0.033226,15.8204,23.2496,430.48,47.372025,8.516394
...,...,...,...,...,...,...,...,...,...,...,...
349,349,2016-04-07 14:54:13.000,681427.711664,247230.716652,64.413921,0.048839,12.8960,43.8402,451.72,47.370885,8.516733
350,350,2016-04-07 14:54:18.000,681423.602691,247235.645341,63.864470,0.061871,12.8960,43.7522,445.42,47.370930,8.516680
351,351,2016-04-07 14:54:23.000,681417.056496,247241.115050,61.756410,0.049484,12.8960,43.7838,462.78,47.370980,8.516594
352,352,2016-04-07 14:54:28.000,681408.562136,247240.850127,63.162392,0.082968,12.8960,43.8590,478.54,47.370978,8.516481


In [10]:
df['Datetime']=pd.to_datetime(df['time']).dt.tz_localize('UTC')
# Add this line to convert from UTC to another timezone (i.e. Australian one)
#df['Australian_datetime']=pd.to_datetime(df['time']).dt.tz_localize('UTC').dt.tz_convert('Australia/Sydney')
df.index=df['Datetime']
df = df.drop('X',axis=1)
df = df.drop('Y',axis=1)
df

,Unnamed: 0,time,Sound,Dust,TempEN,RH,Light,Latitude,Longitude,Datetime
Datetime,,,,,,,,,,
2016-04-07 14:25:13.250000+00:00,0,2016-04-07 14:25:13.250,48.328609,0.033226,15.8204,23.2496,430.48,47.372025,8.516394,2016-04-07 14:25:13.250000+00:00
2016-04-07 14:25:13+00:00,1,2016-04-07 14:25:13.000,48.328609,0.033226,15.8204,23.2496,430.48,47.372025,8.516394,2016-04-07 14:25:13+00:00
2016-04-07 14:25:18+00:00,2,2016-04-07 14:25:18.000,48.328609,0.033226,15.8204,23.2496,430.48,47.372025,8.516394,2016-04-07 14:25:18+00:00
2016-04-07 14:25:23+00:00,3,2016-04-07 14:25:23.000,48.328609,0.033226,15.8204,23.2496,430.48,47.372025,8.516394,2016-04-07 14:25:23+00:00
2016-04-07 14:25:28+00:00,4,2016-04-07 14:25:28.000,48.328609,0.033226,15.8204,23.2496,430.48,47.372025,8.516394,2016-04-07 14:25:28+00:00
...,...,...,...,...,...,...,...,...,...,...
2016-04-07 14:54:13+00:00,349,2016-04-07 14:54:13.000,64.413921,0.048839,12.8960,43.8402,451.72,47.370885,8.516733,2016-04-07 14:54:13+00:00
2016-04-07 14:54:18+00:00,350,2016-04-07 14:54:18.000,63.864470,0.061871,12.8960,43.7522,445.42,47.370930,8.516680,2016-04-07 14:54:18+00:00
2016-04-07 14:54:23+00:00,351,2016-04-07 14:54:23.000,61.756410,0.049484,12.8960,43.7838,462.78,47.370980,8.516594,2016-04-07 14:54:23+00:00


In [11]:
df = df.resample('1s').mean().fillna(method='bfill').fillna(method='ffill')

In [24]:
#df['UserID']=participant_ID

In [12]:
speed_df = df.copy()
speed_df['Session']=str(speed_df.index[0])
speed_df_processed = basic_speed_functions(speed_df)
speed_df_processed

2016-04-07 14:25:13+00:00 possible bike 0.0


,Unnamed: 0,Sound,Dust,TempEN,RH,Light,Latitude,Longitude,Session,Distance,Time_Passed,Speed,Speed_smoother,Activity_type,Moving,Speed_dif,Speed_std,Speed_mean_30sec
Datetime,,,,,,,,,,,,,,,,,,
2016-04-07 14:25:13+00:00,0.5,48.328609,0.033226,15.8204,23.2496,430.48,47.372025,8.516394,2016-04-07 14:25:13+00:00,0.00,0.0,0.1,0.0,Bike,1,0.00,0.309596,-0.105556
2016-04-07 14:25:14+00:00,2.0,48.328609,0.033226,15.8204,23.2496,430.48,47.372025,8.516394,2016-04-07 14:25:13+00:00,0.00,1.0,-0.5,0.0,Bike,1,0.00,0.309596,-0.105556
2016-04-07 14:25:15+00:00,2.0,48.328609,0.033226,15.8204,23.2496,430.48,47.372025,8.516394,2016-04-07 14:25:13+00:00,0.00,1.0,-0.7,0.0,Bike,1,-0.22,0.309596,-0.105556
2016-04-07 14:25:16+00:00,2.0,48.328609,0.033226,15.8204,23.2496,430.48,47.372025,8.516394,2016-04-07 14:25:13+00:00,0.00,1.0,-0.7,0.0,Bike,1,0.08,0.309596,-0.105556
2016-04-07 14:25:17+00:00,2.0,48.328609,0.033226,15.8204,23.2496,430.48,47.372025,8.516394,2016-04-07 14:25:13+00:00,0.00,1.0,-0.4,0.0,Bike,1,0.27,0.309596,-0.105556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-04-07 14:54:29+00:00,353.0,60.623706,0.044753,12.9056,44.0356,527.44,47.371013,8.516389,2016-04-07 14:25:13+00:00,7.94,1.0,5.7,4.5,Bike,1,-0.64,0.784213,5.503226
2016-04-07 14:54:30+00:00,353.0,60.623706,0.044753,12.9056,44.0356,527.44,47.371013,8.516389,2016-04-07 14:25:13+00:00,0.00,1.0,4.7,4.4,Bike,1,-0.97,0.784213,5.503226
2016-04-07 14:54:31+00:00,353.0,60.623706,0.044753,12.9056,44.0356,527.44,47.371013,8.516389,2016-04-07 14:25:13+00:00,0.00,1.0,3.5,4.3,Bike,1,-1.21,0.000000,0.000000


In [13]:
if 'UserID' not in speed_df_processed.columns:
    speed_df_processed['UserID']=participant_ID

In [14]:
speed_df_processed

,Unnamed: 0,Sound,Dust,TempEN,RH,Light,Latitude,Longitude,Session,Distance,Time_Passed,Speed,Speed_smoother,Activity_type,Moving,Speed_dif,Speed_std,Speed_mean_30sec,UserID
Datetime,,,,,,,,,,,,,,,,,,,
2016-04-07 14:25:13+00:00,0.5,48.328609,0.033226,15.8204,23.2496,430.48,47.372025,8.516394,2016-04-07 14:25:13+00:00,0.00,0.0,0.1,0.0,Bike,1,0.00,0.309596,-0.105556,07
2016-04-07 14:25:14+00:00,2.0,48.328609,0.033226,15.8204,23.2496,430.48,47.372025,8.516394,2016-04-07 14:25:13+00:00,0.00,1.0,-0.5,0.0,Bike,1,0.00,0.309596,-0.105556,07
2016-04-07 14:25:15+00:00,2.0,48.328609,0.033226,15.8204,23.2496,430.48,47.372025,8.516394,2016-04-07 14:25:13+00:00,0.00,1.0,-0.7,0.0,Bike,1,-0.22,0.309596,-0.105556,07
2016-04-07 14:25:16+00:00,2.0,48.328609,0.033226,15.8204,23.2496,430.48,47.372025,8.516394,2016-04-07 14:25:13+00:00,0.00,1.0,-0.7,0.0,Bike,1,0.08,0.309596,-0.105556,07
2016-04-07 14:25:17+00:00,2.0,48.328609,0.033226,15.8204,23.2496,430.48,47.372025,8.516394,2016-04-07 14:25:13+00:00,0.00,1.0,-0.4,0.0,Bike,1,0.27,0.309596,-0.105556,07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-04-07 14:54:29+00:00,353.0,60.623706,0.044753,12.9056,44.0356,527.44,47.371013,8.516389,2016-04-07 14:25:13+00:00,7.94,1.0,5.7,4.5,Bike,1,-0.64,0.784213,5.503226,07
2016-04-07 14:54:30+00:00,353.0,60.623706,0.044753,12.9056,44.0356,527.44,47.371013,8.516389,2016-04-07 14:25:13+00:00,0.00,1.0,4.7,4.4,Bike,1,-0.97,0.784213,5.503226,07
2016-04-07 14:54:31+00:00,353.0,60.623706,0.044753,12.9056,44.0356,527.44,47.371013,8.516389,2016-04-07 14:25:13+00:00,0.00,1.0,3.5,4.3,Bike,1,-1.21,0.000000,0.000000,07


In [15]:
# Put here the name of the directory where you want to save the resulting files
path_to_save = r'C:\Users\demdr\Desktop\Testing the thesis functions\Project data\Analysed data\Participants\ID\Speed data\Session 1.csv'
path_to_save = path_to_save.replace('ID',participant_ID)
for_final_df = speed_df_processed.fillna(method='bfill').fillna(method='ffill').fillna(0).copy()
for_final_df.to_csv(path_to_save)